# Analiza podatkov

Analizirali bomo podatke iz hribi.net. Analizirali bomo:

- Gore
- Poti

In [46]:
import pandas as pd

pd.set_option("display.max_rows", 150)

gore = pd.read_csv("gore.csv", index_col="id")

Oglejmo si podatke.

In [47]:
gore

,ime,država,gorovje,višina,"geografska širina, dolžina",vrsta,število ogledov,priljubljenost,mesto na lestvici priljubljenosti,število slik,število poti,število GPS sledi,opis
id,,,,,,,,,,,,,
101,Triglav,Slovenija,Julijske Alpe,2864,"46,37832°N 13,83648°E","vrh, bivak",1.699.319,100%,1.0,42,28,20,"Triglav je 2864 m visoka gora, ki se nahaja v ..."
1010,Dolkova špica,Slovenija,Julijske Alpe,2591,"46,42574°N 13,81467°E",vrh,579.592,95%,170.0,17,5,2,"Dolkova špica je 2591 m visoka gora, ki se nah..."
10100,Koča na Doliču,Slovenija,Julijske Alpe,2151,"46,36487°N 13,81953°E","koča, sedlo",631.697,96%,124.0,10,10,1,Koča na Doliču (včasih Tržaška koča na Doliču)...
10101,Planina Krstenica,Slovenija,Julijske Alpe,1670,"46,32596°N 13,85037°E",planina,534.805,92%,258.0,11,3,2,"Planina Krstenica je razgledna planina, ki se ..."
10102,Dom na Komni,Slovenija,Julijske Alpe,1520,"46,28446°N 13,77338°E",koča,810.196,99%,44.0,10,12,2,"Dom, ki je odprt vse leto se nahaja na neizraz..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7701660,Golem Korab / Maja e Korabit,"Albanija, Severna Makedonija",Jablanica,2764,"41,79013°N 20,54709°E",vrh,274.753,68%,1100.0,6,1,0,Izredno lahko dostopen najvišji vrh Makedonije...
7801563,Rysy,"Poljska, Slovaška",Visoke Tatre,2503,"49,17929°N 20,08846°E",vrh,317.009,73%,942.0,14,2,0,Rysy je vrh v Visokih Tatrah na meji med Slova...
7802730,Slavkovsky štit,Slovaška,Visoke Tatre,2452,"49,1659°N 20,1846°E",vrh,4.000,35%,2221.0,1,1,0,Slavkovsky štit je četrti najvišji vrh v Visok...


Oglejmo si 15 najvišjih slovenskih vrhov.

In [56]:
slovenske_gore = gore[gore["država"].str.contains("Slovenija")].copy().sort_values("višina", ascending=False)
slovenske_gore[["ime", "višina"]].head(15)

,ime,višina
id,,
101,Triglav,2864
102,Škrlatica,2740
103,Mali Triglav,2725
104,Mangart,2679
105,Visoki Rokav,2646
106,Jalovec,2645
107,Veliki Oltar,2621
108,Velika Martuljška Ponca,2602
109,Razor,2601


In [61]:
slovenski_dvatisocaki = slovenske_gore[slovenske_gore["višina"] >= 2000]
slovenski_dvatisocaki

,ime,država,gorovje,višina,"geografska širina, dolžina",vrsta,število ogledov,priljubljenost,mesto na lestvici priljubljenosti,število slik,število poti,število GPS sledi,opis
id,,,,,,,,,,,,,
101,Triglav,Slovenija,Julijske Alpe,2864,"46,37832°N 13,83648°E","vrh, bivak",1.699.319,100%,1.0,42,28,20,"Triglav je 2864 m visoka gora, ki se nahaja v ..."
102,Škrlatica,Slovenija,Julijske Alpe,2740,"46,43277°N 13,82114°E",vrh,828.423,99%,39.0,23,5,5,Škrlatica je druga najvišja gora v Sloveniji. ...
103,Mali Triglav,Slovenija,Julijske Alpe,2725,"46,37683°N 13,84315°E",vrh,531.498,92%,265.0,4,3,2,Mali Triglav je neizrazit vrh v glavnem greben...
104,Mangart,"Italija, Slovenija",Julijske Alpe,2679,"46,43948°N 13,65457°E",vrh,965.176,99%,18.0,26,8,5,"Mangart (tudi Mangrt, obe zapisani obliki sta ..."
105,Visoki Rokav,Slovenija,Julijske Alpe,2646,"46,43547°N 13,8283°E",vrh,523.449,92%,280.0,2,2,0,Visoki Rokav je najvišja gora v Sloveniji na k...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10172,Rjavo jezero,Slovenija,Julijske Alpe,2006,"46,35541°N 13,79558°E",jezero,452.851,87%,463.0,8,4,0,Rjavo jezero je najvišje od vseh Triglavskih j...
10461,Ablanca,Slovenija,Julijske Alpe,2004,"46,35294°N 13,88419°E",vrh,547.928,93%,234.0,11,9,1,"Ablanca je 2004 m visok vrh, ki se nahaja jugo..."
10325,Lanževica,Slovenija,Julijske Alpe,2003,"46,29681°N 13,72491°E",vrh,511.451,91%,304.0,14,6,4,"Lanževica je 2003 m visok vrh, ki se nahaja me..."
